In [1]:
import os
import re
import nltk
from tqdm import tqdm
import pandas as pd

nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/eljan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def clean_text(text):
    # Replace new lines with spaces
    cleaned_text = re.sub(r'\n', ' ', text)
    # Remove multiple spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.strip()

def chunk_text(text, max_words=1000, overlap=1):
    sentences = nltk.sent_tokenize(text)
    current_count = 0
    current_text = []
    splits = []
    for i, sentence in tqdm(enumerate(sentences)):
        words = nltk.word_tokenize(sentence)
        if current_count + len(words) > max_words and current_text:
            splits.append(" ".join(current_text))
            current_text = current_text[-overlap:]  # Overlap the last sentence(s)
            current_count = sum(len(nltk.word_tokenize(s)) for s in current_text)
        current_text.append(sentence)
        current_count += len(words)
    # Add the last split if it has content
    if current_text:
        splits.append(" ".join(current_text))
    return splits

def process_file(file_path, chunk_size, overlap):
    text = read_file(file_path)
    cleaned_text = clean_text(text)
    chunks = chunk_text(cleaned_text, chunk_size, overlap)
    return chunks

def process_directory(directory_path, chunk_size, overlap):
    file_chunks = {}
    for filename in tqdm(os.listdir(directory_path), desc="Processing files"):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory_path, filename)
            chunks = process_file(file_path, chunk_size, overlap)
            file_chunks[filename] = chunks
    return file_chunks

directory_path = 'texts'
chunk_size = 512
overlap = 1

file_chunks = process_directory(directory_path, chunk_size, overlap)

8407it [00:00, 29843.39it/s]     | 0/24 [00:00<?, ?it/s]
141939it [00:06, 23257.98it/s]   | 1/24 [00:00<00:08,  2.74it/s]
28002it [00:01, 26507.28it/s]    | 3/24 [00:07<01:00,  2.88s/it]
138981it [00:05, 26702.92it/s]   | 4/24 [00:09<00:46,  2.33s/it]
124266it [00:05, 24837.54it/s]   | 5/24 [00:15<01:10,  3.69s/it]
53435it [00:02, 25593.56it/s]    | 6/24 [00:21<01:20,  4.47s/it]
1252792it [00:48, 26029.09it/s]  | 7/24 [00:24<01:05,  3.88s/it]
49459it [00:01, 26737.71it/s]    | 8/24 [01:24<05:39, 21.24s/it]
2373082it [01:20, 29484.75it/s]  | 9/24 [01:26<03:51, 15.42s/it]
34216it [00:01, 21430.69it/s]    | 10/24 [03:07<09:40, 41.45s/it]
74519it [00:03, 24533.10it/s]    | 11/24 [03:09<06:22, 29.46s/it]
1378157it [00:54, 25338.14it/s]  | 12/24 [03:12<04:20, 21.68s/it]
33502it [00:01, 26077.84it/s]    | 13/24 [04:20<06:30, 35.47s/it]
2547965it [02:12, 19267.29it/s]  | 14/24 [04:21<04:12, 25.27s/it]
28059it [00:01, 26471.58it/s]▎   | 15/24 [07:07<10:08, 67.58s/it]
8322it [00:00, 28525.13it/s

In [63]:
df.shape

(317544, 2)

: 

In [41]:
data = [(key, value) for key, values in file_chunks.items() for value in values]
df = pd.DataFrame(data, columns=['source', 'text'])
df = df[['text', 'source']]
df['source'] = df['source'].apply(lambda x: x.split('.')[0])
df.to_parquet('chunks.parquet', index=False)

In [61]:
df.shape

(317544, 2)

In [62]:
df

,text,source
0,"Orta ixtisas təhsili müəssisələrinin tələbələri üçün dərslik Əliyeva Fərqanə Aydın qızı Daxili İşlər Nazirliyinin Polis Akademiyasının üzrə fəlsəfə doktoru Hüseynov Mərdan Həsən oğlu Azərbaycan Kooperasiya Universitetinin baş müəllimi, Pedaqogika üzrə fəlsəfə doktoru, dosent Məmmədova Vüsalə Əhməd qızı Azərbaycan Texniki Universiteti nəzdində Bakı Dövlət Rabitə və Nəqliyyat Kollecinin müəllimi Azərbaycan Texniki Universiteti nəzdində Bakı Dövlət Rabitə və Nəqliyyat Kollecinin müəllimi Elmi Redaktor: Hüseynov Mərdan Həsən oğlu Azərbaycan Kooperasiya Universitetinin baş müəllimi, Pedaqogika üzrə fəlsəfə doktoru, dosent Redaktor: Hacıyeva Sevda Rauf qızı Azərbaycan Texniki Universiteti nəzdində Bakı Dövlət Rabitə və Nəqliyyat Kollecinin müəllimi Salayeva, Abdinova "" Etika və psixologiya "" : Orta ixtisas təhsili müəssisələrinin tələbələri üçün dərslik. səhifə, şəkilli. Çağdaş dövrə xas olan mənəvi problematikanın aktuallığı onun tədqiqinin bütün aspektlərinə marağı artırır. Bu, mürəkkəb dünyanın bəzi qanunauyğunluqlarını hələ də qəbul edə bilməyənlərə, müəyyən çətinliklərlə qarşılaşanda onlarla necə mübarizə aparıb aradan qaldırmağın yollarını tapmağa çalışanlara və ən əsası, bu günün şagirdlərinə və sabahın mütəxəssislərinə kömək edə biləcək biliklər verməkdir. Elm və texnikanın sürətlə inkişaf etdiyi bir dövrdə yaşayırıq. Bu dövr yeni elm sahələrinin meydana gəlməsi ilə fərqlənən bir dövrdür. Özünün qədim tarixiliyi ilə fərqlənən elm sahələri ilə yanaşı tamamilə yeni elmlərin meydana gəlməsi onları bir-birindən fərqləndirməyi, onlardan yerliyerində bəhrələnməyi tələb edir. Bu elmlərin bir çoxunun obyekti eyni olduğu halda, onlar öz predmetlərinə görə birbirindən fərqlənirlər. Hər bir elmin predmetini özünəməxsus hadisələr təşkil edir. Bu hadisələrdən bəziləri olduqca mürəkkəb, çətin dərk olunan hadisələrdir. İnsan şəxsiyyətinin xüsusiyyətləri, adamlarla ünsiyyət qurmaq bacarığı, hər bir insanın ürəyinə yol tapmaq, iş yerində davranış qaydaları haqda biliklər müasir insana lazım olan vacib biliklərdir. Dünyanın mahiyyətini anlamağa çalışan filosoflar fəlsəfənin ənənəvi problemləri olan "" dünya necə yaranıb "", ilə yanaşı, "" insanın dünyada mövqeyi "", "" insanın həyatı "", "" cəmiyyətdə fəaliyyəti "", "" ictimai mahiyyəti "", "" birgəyaşayış normaları "" kimi həyat əhəmiyyətli suallara da cavab axtarmağa başladılar. Artıq filosoflar yalnız obyekt-subyekt münasibətləri ilə deyil, həm də subyekt-subyekt münasibətləri ilə də maraqlanırdılar. Bu problemin sürətli inkişafı əsrdən, Sokratın dövründən başlayıb. Bu gün tədrisi vacib fənlərdən olan "" Etika və psixologiya "" antik dövrdən müasir dövrümüzədək uzun bir tarixi yol keçmişdir. Hal-hazırda baş verən ictimai proseslər, insanın mənəvi inkişafı və s. "" Etika və psixologiya "" nın tədqiqat obyektindən kənarda deyil. Bu baxımdan "" Etika və psixologiya "" gənclərin şəxsiyyət kimi formalaşmasında, onların əxlaqi şüurunun inkişafında mühüm rol oynayan fənlərdəndir. Dərslik insanlar arasında mövcud olan davranış qaydalarının, əxlaq normalarının tənzim edilməsi ideyasını aşılayır. Etika elmi haqqında 1. Müxtəlif dövrlərdə etik fikir. Azərbaycanda etik fikrin tarixi 1. Müsəlman etikası II Fəsil. Əxlaqın mahiyyəti və əsas funksiyaları 2. Əxlaqın mahiyyəti 2. İctimai şüurun formaları 2. Əxlaqi fəaliyyət III Fəsil. Etikanın əsas kateqoriyaları 3. Xeyir və şər 3. Borc və vicdan 3. Şərəf və ləyaqət 3. Xoşbəxtlik və həyatın mənası IV Fəsil.",psixologiya
1,"Xoşbəxtlik və həyatın mənası IV Fəsil. Peşə etikası 4. Peşə etikasının inkişaf tarixi 4. İşgüzar etikanın əhəmiyyəti 4. Əmək kollektivində işgüzar münasibətlər etikası qaydaları 5. Davranış mədəniyyəti və etiket 5. Salamlaşma və danışıq mədəniyyəti 5. Xidmət sahələrində davranış və ünsiyyət mədəniyyəti 5. İctimai yerlərdə davranış qaydaları VI. Psixologiyanın ümumi əsasları 6. Psixologiya elmi haqqında 6. İnsan, şəxsiyyət və fərdiyyət 6. Ünsiyyət haqqında anlayış 6. Fəaliyyət və fəallıq VII. Psixi hadisələr. Qabiliyyətlə

In [59]:
from simhash import Simhash

def get_features(text):
    # Simple word tokenization; consider more sophisticated methods for real applications
    words = text.lower().split()
    return words

def compute_simhash(text):
    features = get_features(text)
    return Simhash(features)

# Example texts
text1 = "This is a sample text for deduplication man."
text2 = "This is a sample text for deduplication."
text3 = "This is another example of a text that might be similar."
text4 = "Hello  man how are you doing"

# Compute SimHashes
hash1 = compute_simhash(text1)
hash2 = compute_simhash(text2)
hash3 = compute_simhash(text3)
hash4 = compute_simhash(text4)

# Comparing hashes
print(f"Hash1 vs Hash2: {hash1.distance(hash2)}")  # Lower score -> more similar
print(f"Hash1 vs Hash3: {hash1.distance(hash3)}")
print(f"{hash1.distance(hash4)}")


Hash1 vs Hash2: 10
Hash1 vs Hash3: 18
26
